In [ ]:
import facebook
import requests as r
import csv
import json
from bs4 import BeautifulSoup

token='ADD_ACCESS_TOKEN_HERE'

graph = facebook.GraphAPI(access_token=token, version = 2.7)

In [ ]:
posts = graph.request('/csu/posts')
posts

In [ ]:
german_parties = ['cdu', 
                  'spd', 
                  'linkspartei', 
                  'b90diegruenen',
                  'csu',
                  'fdp',
                  'alternativefuerde',
                  'freie.waehler.bundesvereinigung']

In [ ]:
# Get non-political advertisers
reader = csv.reader(open('german_ads.csv', 'r'))

german_advertisers = set()
for line in reader:
    if line[2] == '0':
        soup = BeautifulSoup(line[1])
        for link in soup.find_all('a', href=True):
            if 'facebook.com' in link['href']:
                name = link['href'].split('/')[3]
                if '-' in name:
                    name = name.split('-')[-1]
                german_advertisers.add(name)
                break         
            

In [ ]:
def get_posts_by_page(page_name, num_pages):
    try:
        posts = graph.request('/'+page_name+'/posts')
    except:
        return []
    page_count = 0
    post_bodies = []
    while posts and page_count < num_pages:
        for post in posts['data']:
            if 'message' in post:
                post_bodies.append(post['message'])
        if page_count < num_pages and 'paging' in posts :
            posts = r.get(posts['paging']['next']).json()
            page_count += 1
        else :
            posts = None
            break
    return post_bodies


In [ ]:
posts_by_party = dict()

for party in german_parties:
    posts_by_party[party] = get_posts_by_page(party, 8)

In [ ]:
posts_by_non_political = dict()
for advertiser in german_advertisers:
    posts_by_non_political[advertiser] = get_posts_by_page(advertiser, 8)

In [ ]:
political_messages = [x.replace('\n', ' ') for party in posts_by_party for x in posts_by_party[party]]
non_political_messages = [x.replace('\n', ' ') for advertiser in posts_by_non_political for x in posts_by_non_political[advertiser]]

In [ ]:
non_political_messages

In [ ]:
json.dump(political_messages, open('political_german.json', 'w'))
json.dump(non_political_messages, open('not_political_german.json', 'w'))